In [6]:
url_ipca_completo = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
url_ipca = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
# url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
indices_mensais = {}

response = requests.get(url_ipca_completo)
if response.status_code == 200:
    print('Requisição realizada com sucesso!')
response.raise_for_status()

dados_ipca = json.loads(response.content)
indices_ipca = dados_ipca[0]['resultados'][0].get('series')[0].get('serie')
print(f'{len(indices_ipca)} índices IPCA mensais extraídos do IBGE')

# Cria uma lista de tuplas a partir dos itens do dicionário
lista_dados = list(indices_ipca.items())

# Cria o DataFrame
df = pd.DataFrame(lista_dados, columns=["AnoMes", "Valor"])

Requisição realizada com sucesso!
534 índices IPCA mensais extraídos do IBGE


In [7]:
df

,AnoMes,Valor
0,197912,0.0000000076183
1,198001,0.0000000081223
2,198002,0.0000000084973
3,198003,0.0000000090104
4,198004,0.0000000094867
...,...,...
529,202401,6801.7200000000000
530,202402,6858.1700000000000
531,202403,6869.1400000000000
532,202404,6895.2400000000000


In [2]:
# dados_ipca

In [9]:
import altair as alt
import pandas as pd
import requests
import pandas as pd
import json
import csv
from io import BytesIO, TextIOWrapper

def plotar_perdas(ano_inicio, ano_final, reajustes_anuais):
    url_ipca_completo = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
    url_ipca = "https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/2266?localidades=N1[all]"
    # url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
    indices_mensais = {}

    response = requests.get(url_ipca_completo)
    if response.status_code == 200:
        print('Requisição realizada com sucesso!')
    response.raise_for_status()

    dados_ipca = json.loads(response.content)
    indices_ipca = dados_ipca[0]['resultados'][0].get('series')[0].get('serie')
    print(f'{len(indices_ipca)} índices IPCA mensais extraídos do IBGE')

    # Cria uma lista de tuplas a partir dos itens do dicionário
    lista_dados = list(indices_ipca.items())

    # Cria o DataFrame
    df = pd.DataFrame(lista_dados, columns=["AnoMes", "Valor"])    

    # 1. Converter a coluna `Valor` para o tipo numérico
    df['Valor'] = pd.to_numeric(df['Valor'])

    # # Filtrar o DataFrame para o período desejado
    # df = df[
    #     (int(str(df['AnoMes'])[:4]) >= ano_inicio) & (int(str(df['AnoMes'])[:4]) <= ano_final)
    # ]

    # 2. Criar uma nova coluna `Ano` extraindo o ano da coluna `AnoMes`
    df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

    # 3. Calcular a variação percentual mensal do IPCA
    df['InflacaoMensal'] = df['Valor'].pct_change() * -100
    df['InflacaoMensal'] = df['InflacaoMensal'].fillna(0)

    # 4. Calcular a inflação acumulada por ano
    df_inflacao_anual = df.groupby('Ano')['InflacaoMensal'].sum().reset_index()
    df_inflacao_anual = df_inflacao_anual.rename(columns={'InflacaoMensal': 'PerdaInflacao'})

    # 5. Criar um dicionário com anos e correções
    anos = df_inflacao_anual['Ano'].unique().tolist()
    correcoes_salariais_lista = [0 for x in anos]  # Lista de zeros por padrão
    if reajustes_anuais:  # Verificar se a lista de reajustes não está vazia
        correcoes_salariais_lista = reajustes_anuais[:len(anos)]  # Usar os reajustes fornecidos
    correcoes_salariais = dict(zip(anos, correcoes_salariais_lista))

    # 6. Criar um DataFrame a partir das correções salariais
    df_salarios = pd.DataFrame(list(correcoes_salariais.items()), columns=['Ano', 'CorrecaoSalarial'])

    # 7. Fazer o merge entre os DataFrames
    df_merged = df_inflacao_anual.merge(df_salarios, on='Ano', how='left')

    # 8. Preencher os valores NaN da coluna 'CorrecaoSalarial' com 0
    df_merged['CorrecaoSalarial'] = df_merged['CorrecaoSalarial'].fillna(0)

    # 9. Calcular a diferença acumulada entre a correção salarial e a perda por inflação
    df_merged['SaldoAcumulado'] = (df_merged['CorrecaoSalarial'] - df_merged['PerdaInflacao']).cumsum()

    # Filtrar o DataFrame para o período desejado
    df_merged_filtrado = df_merged[
        (df_merged['Ano'] >= ano_inicio) & (df_merged['Ano'] <= ano_final)
    ]

    # Zerar o saldo acumulado no ano de início
    df_merged_filtrado.loc[df_merged_filtrado['Ano'] == ano_inicio, 'SaldoAcumuladoAjustado'] = 0

    # 10. Calcular o valor absoluto máximo para definir o domínio do eixo y principal e secundário
    max_abs_valor = max(
        df_merged_filtrado['PerdaInflacao'].abs().max(),
        df_merged_filtrado['CorrecaoSalarial'].abs().max(),
        df_merged_filtrado['SaldoAcumulado'].abs().max()
    )

    # Preparar os dados para o gráfico de barras
    df_barras = df_merged_filtrado.melt(id_vars='Ano', value_vars=['PerdaInflacao', 'CorrecaoSalarial'], var_name='Tipo', value_name='Valor')

    # Calcular o valor absoluto máximo para definir o domínio do eixo y principal
    max_abs_valor_barras = max(df_merged_filtrado['PerdaInflacao'].abs().max(), df_merged_filtrado['CorrecaoSalarial'].abs().max())

    # Calcular o valor mínimo e máximo do SaldoAcumulado para definir o domínio do eixo y da linha
    min_saldo_acumulado = df_merged_filtrado['SaldoAcumulado'].min()
    max_saldo_acumulado = df_merged_filtrado['SaldoAcumulado'].max()

    # Criar o gráfico de barras
    barras = alt.Chart(df_barras).mark_bar().encode(
        x=alt.X('Ano:O', axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('Valor:Q', axis=alt.Axis(title='Variação (%)', titleColor='black'), scale=alt.Scale(domain=[-max_abs_valor_barras, max_abs_valor_barras])),
        color=alt.Color('Tipo:N', scale={'domain': ['PerdaInflacao', 'CorrecaoSalarial'], 'range': ['red', 'steelblue']}),
        tooltip=['Ano', 'Valor', 'Tipo']
    ).properties(
        title='Perda por Inflação, Reposição Salarial e Saldo Acumulado'
    )

    # Adicionar os rótulos às barras
    text = barras.mark_text(
        align='center',
        baseline='bottom',
        dy=-5
    ).encode(
        text=alt.Text('Valor:Q', format='.2f')
    )

    # Criar o gráfico de linha para o saldo acumulado (eixo secundário)
    linha = alt.Chart(df_merged_filtrado).mark_line(color='green', point=True).encode(
        x='Ano:O',
        y=alt.Y('SaldoAcumulado:Q', axis=alt.Axis(title='Saldo Acumulado (%)', titleColor='green'), scale=alt.Scale(domain=[min_saldo_acumulado, max_saldo_acumulado])),
        tooltip=['Ano', 'SaldoAcumulado']
    )

    # Adicionar rótulos de dados à linha
    text_linha = linha.mark_text(
        align='left',
        baseline='middle',
        dx=5,  # Nudge text to right so it doesn't overlap with the line
        color='green'
    ).encode(
        text=alt.Text('SaldoAcumulado:Q', format='.2f')
    )

    # Combinar os gráficos com os rótulos
    grafico_final = alt.layer(barras + text, linha + text_linha).resolve_scale(
        y='independent'
    ).interactive()


    grafico_final = grafico_final.properties(
        height=600,  # Largura de 800 pixels
        width=1100  # Largura de 1200 pixels
    )

    return df_merged, grafico_final

# Exemplo de uso
ano_inicio = 2003
ano_final = 2023
reajustes_anuais = [2.5, 3.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5]  # Exemplo de reajustes

df_merged, grafico_final = plotar_perdas(ano_inicio, ano_final, reajustes_anuais)
grafico_final

Requisição realizada com sucesso!
534 índices IPCA mensais extraídos do IBGE


C:\Users\marco\AppData\Local\Temp\ipykernel_10248\3423049043.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_filtrado.loc[df_merged_filtrado['Ano'] == ano_inicio, 'SaldoAcumuladoAjustado'] = 0


alt.LayerChart(...)

In [17]:
import altair as alt
alt.renderers.enable('jupyterlab')
grafico_final

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [ ]:
# Converter a coluna `Valor` para o tipo numérico
df['Valor'] = pd.to_numeric(df['Valor'])

# Extrair o ano da coluna `AnoMes`
df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

# Filtrar para manter apenas os meses de dezembro
df_dezembro = df[df['AnoMes'].astype(str).str[-2:] == '12'].copy()

# Agrupar por ano e selecionar o primeiro valor de cada ano
df_anual = df_dezembro.groupby('Ano')['Valor'].first().reset_index()

# Obter o ano de início e o ano de término do usuário
ano_inicio = int(input("Digite o ano de início: "))
ano_termino = int(input("Digite o ano de término: "))

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

In [ ]:
# Converter a coluna `Valor` para o tipo numérico
df['Valor'] = pd.to_numeric(df['Valor'])

# Extrair o ano da coluna `AnoMes`
df['Ano'] = df['AnoMes'].astype(str).str[:4].astype(int)

# Filtrar para manter apenas os meses de dezembro
df_dezembro = df[df['AnoMes'].astype(str).str[-2:] == '12'].copy()

# Agrupar por ano e selecionar o primeiro valor de cada ano
df_anual = df_dezembro.groupby('Ano')['Valor'].first().reset_index()

# Obter o ano de início e o ano de término do usuário
ano_inicio = int(input("Digite o ano de início: "))
ano_termino = int(input("Digite o ano de término: "))

# Filtrar para o período desejado
df_periodo = df_anual[(df_anual['Ano'] >= ano_inicio) & (df_anual['Ano'] <= ano_termino)]

# Calcular a variação acumulada do IPCA
variacao_acumulada = (df_periodo['Valor'].iloc[-1] / df_periodo['Valor'].iloc[0] - 1) * 100

# Calcular a taxa anual equivalente
num_anos = ano_termino - ano_inicio + 1
taxa_anual = ((1 + variacao_acumulada / 100) ** (1 / num_anos) - 1) * 100

# Imprimir os resultados
print(f'Variação acumulada do IPCA de {ano_inicio} a {ano_termino}: {variacao_acumulada:.2f}%')
print(f'Taxa anual equivalente: {taxa_anual:.2f}%')

In [ ]:
import altair as alt

# Criar um DataFrame com os dados de reposição salarial
df_reposicao = df_merged[['Ano', 'CorrecaoSalarial']].rename(columns={'CorrecaoSalarial': 'Valor'})
df_reposicao['Tipo'] = 'Reposição Salarial'

# Criar um DataFrame com os dados de perda por inflação
df_perda_inflacao = df_merged[['Ano', 'InflacaoPercentual']].rename(columns={'InflacaoPercentual': 'Valor'})
df_perda_inflacao['Valor'] = -df_perda_inflacao['Valor']  # Inverter os valores para ficarem negativos
df_perda_inflacao['Tipo'] = 'Perda por Inflação'

# Concatenar os DataFrames de reposição e perda
df_plot = pd.concat([df_reposicao, df_perda_inflacao])

# Calcular o valor absoluto máximo para definir o domínio do eixo y principal
max_abs_valor = df_plot['Valor'].abs().max()

# Criar o gráfico de barras
barras = alt.Chart(df_plot).mark_bar().encode(
    x=alt.X('Ano:O', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Valor:Q', scale=alt.Scale(domain=[-max_abs_valor, max_abs_valor])),  # Ajustar escala y
    color=alt.Color('Tipo:N', scale={'domain': ['Perda por Inflação', 'Reposição Salarial'], 'range': ['red', 'steelblue']}),
    tooltip=['Ano', 'Valor', 'Tipo']
).properties(
    title='Reposição Salarial, Perda por Inflação e Saldo Acumulado'
)

# Adicionar os rótulos às barras
text = barras.mark_text(
    align='center',
    baseline='bottom',
    dy=-5  # Nudge text upwards so it doesn't overlap with the bar
).encode(
    text=alt.Text('Valor:Q', format='.2f')
)

# Criar o gráfico de linha para o saldo acumulado (eixo secundário)
linha = alt.Chart(df_merged).mark_line(color='green', point=True).encode(
    x='Ano:O',
    y=alt.Y('DiferencaAcumulada:Q', axis=alt.Axis(title='Saldo Acumulado', titleColor='green')),
    tooltip=['Ano', 'DiferencaAcumulada']
)

# Combinar os gráficos
grafico_final = alt.layer(barras + text, linha).resolve_scale(
    y = 'independent'
).interactive()

# Exibir o gráfico
grafico_final

grafico_final = grafico_final.properties(
    height=600,  # Largura de 800 pixels
    width=1100  # Largura de 1200 pixels
)
# Salve o gráfico em um arquivo JSON
grafico_final.save('grafico_perda_ganho_poder_compra.json')

In [ ]:
import altair as alt
alt.renderers.enable('jupyterlab')
grafico_final

In [ ]:
df_merged

In [ ]:
from matplotlib import pyplot as plt

plt.plot(indices_ipca.values())

In [ ]:
len(dados_ipca[0])

In [ ]:
import requests
import pandas as pd
import json
import csv
from io import BytesIO, TextIOWrapper

url_tab_var = "https://apisidra.ibge.gov.br/values/t/1737/n1/all/v/2266/p/all"
indices_mensais = {}

response = requests.get(url_tab_var)
response.raise_for_status()

meses=[]
indices=[]

# Ler o CSV com TextIOWrapper para converter bytes em strings
with TextIOWrapper(BytesIO(response.content), encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=";")

    # Extrair dados relevantes e construir dicionário
    for row in reader:
        print(row)
        if "D3C" in row:
            meses.append(row)
        if "V" in row:
            indices.append(row)

In [ ]:
len(meses)

In [ ]:
len(indices)

In [ ]:
# !pip install sidrapy

In [ ]:
import sidrapy

# Importa os dados do SIDRA
ipca_raw = sidrapy.get_table(table_code= "1737",
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            )

# Checamos a importação
ipca_raw.head()

In [ ]:
import pandas as pd

# Converter a coluna D3C para numérico, ignorando valores não numéricos
ipca_raw['D3C'] = pd.to_numeric(ipca_raw['D3C'], errors='coerce')

# Filtrar as linhas onde D3C é igual a 2265
filtro = ipca_raw['D3C'] == 2265
ipca_filtrado = ipca_raw[filtro]

ipca_filtrado[["V","D2C"]]

In [ ]:
import sidrapy

# Importa os dados do SIDRA
pib_sa_raw = sidrapy.get_table(table_code= "1621",
                            territorial_level = "1",
                            ibge_territorial_code = "all",
                            period = "all",
                            classification = "11255/90707")

# Checamos a importação
pib_sa_raw.head()

In [ ]:
from poder_compra import IPCAData

ipca = IPCAData()
meses, indices = ipca.obter_numeros_indice()
for i, j in zip(meses, indices):
    print(i,j)


In [ ]:
indices[:3]

In [ ]:
meses[0]

In [ ]:
import json
a,b = indices[3].split(": ")
{a.replace("\"",""): (b.replace("\"","").replace(",",""))}

In [ ]:
a.replace("\"","")

In [ ]:
float(b.replace("\"","").replace(",",""))

In [ ]:
# Extrair dados relevantes e construir dicionário
for row in reader:
    if row[2] == "MN" and row[1] == "Número-índice":
        data = row[4270].replace(" ", "-") 
        valor = float(row[3].replace(",", "."))
        indices_mensais[data] = valor

In [ ]:
indices_mensais

In [ ]:
from poder_compra import IPCAData

ipca_data = IPCAData()
poder_compra = PoderCompra(ipca_data.dados_ipca)
perdas = poder_compra.calcular_perdas()

previsao = PrevisaoInflacao(ipca_data.dados_ipca)
inflacao_futura = previsao.prever()

Visualizacao.plotar_evolucao_poder_compra(ipca_data.dados_ipca, perdas)